In [1]:
import importlib
import math
import os
import subprocess
import sys


import cvxpy as cp
from cvxpygen import cpg
import scipy as sp
import numpy as np



In [2]:
cp.installed_solvers()

['CLARABEL', 'ECOS', 'ECOS_BB', 'OSQP', 'QOCO', 'SCIPY', 'SCS']

In [3]:
SOLVER = cp.ECOS

In [4]:
N = 10
M = 10
max_assigned = math.ceil(N / M)
np.random.seed(0)
W = sp.stats.lognorm.rvs(s=2, size=(N, M))
W

array([[3.40593534e+01, 2.22624079e+00, 7.08143073e+00, 8.83924358e+01,
        4.18928840e+01, 1.41627379e-01, 6.68707685e+00, 7.38810058e-01,
        8.13476937e-01, 2.27321926e+00],
       [1.33387354e+00, 1.83301458e+01, 4.58172448e+00, 1.27551502e+00,
        2.42959959e+00, 1.94906283e+00, 1.98490897e+01, 6.63440220e-01,
        1.87036837e+00, 1.81193188e-01],
       [6.06039905e-03, 3.69594839e+00, 5.63429708e+00, 2.26654139e-01,
        9.36448324e+01, 5.45448844e-02, 1.09583544e+00, 6.87723989e-01,
        2.14464349e+01, 1.88916030e+01],
       [1.36328176e+00, 2.13043256e+00, 1.69386618e-01, 1.90327721e-02,
        4.98663231e-01, 1.36710852e+00, 1.17116182e+01, 1.10757684e+01,
        4.60863380e-01, 5.46289895e-01],
       [1.22811339e-01, 5.84235700e-02, 3.29573697e-02, 4.94791214e+01,
        3.60845870e-01, 4.16383485e-01, 8.16273648e-02, 4.73499519e+00,
        3.96447918e-02, 6.53455685e-01],
       [1.66804448e-01, 2.16799981e+00, 3.60014750e-01, 9.43009167e-02,
   

In [5]:
x = cp.Variable((N, M), 'x')
w = cp.Parameter((N, M), name='w')
constraints = [
    cp.sum(x, 0) <= max_assigned,  # enforce even distribution
    cp.sum(x, 1) == 1,
    0 <= x,
]
problem = cp.Problem(cp.Minimize(cp.vdot(w, x)), constraints)

In [6]:
code_dir = f"assign_{N}_{M}"
cpg.generate_code(
    problem,
    code_dir=code_dir,
    solver=SOLVER,
    wrapper=False,
)

Generating code with CVXPYgen ...
CVXPYgen finished generating code.


In [7]:
code_dir_abs = os.path.join(os.path.abspath("."), code_dir)
code_dir_abs

'C:\\Users\\volke\\PycharmProjects\\phantom-sc2\\notebooks\\assign_10_10'

In [8]:
sys.executable

'C:\\Users\\volke\\PycharmProjects\\phantom-sc2\\.venv11\\Scripts\\python.exe'

In [9]:
try:
    output = subprocess.check_output(
        f"{sys.executable} setup.py --quiet build_ext --inplace",
        cwd=code_dir,
        shell=True,
    )
except subprocess.CalledProcessError as exc:
    print("Status : FAIL", exc.returncode, exc.output)
else:
    print("Output: \n{}\n".format(output))

Status : FAIL 1 b'-- Selecting Windows SDK version 10.0.22621.0 to target Windows 10.0.26100.\n-- The C compiler identification is MSVC 19.42.34436.0\n-- The CXX compiler identification is MSVC 19.42.34436.0\n-- Detecting C compiler ABI info\n-- Detecting C compiler ABI info - done\n-- Check for working C compiler: C:/Program Files (x86)/Microsoft Visual Studio/2022/BuildTools/VC/Tools/MSVC/14.42.34433/bin/Hostx64/x64/cl.exe - skipped\n-- Detecting C compile features\n-- Detecting C compile features - done\n-- Detecting CXX compiler ABI info\n-- Detecting CXX compiler ABI info - done\n-- Check for working CXX compiler: C:/Program Files (x86)/Microsoft Visual Studio/2022/BuildTools/VC/Tools/MSVC/14.42.34433/bin/Hostx64/x64/cl.exe - skipped\n-- Detecting CXX compile features\n-- Detecting CXX compile features - done\n-- Configuring done (2.6s)\n-- Generating done (0.0s)\n-- Build files have been written to: C:/Users/volke/PycharmProjects/phantom-sc2/notebooks/assign_10_10/c/build\nMSBuil

In [17]:
module = importlib.import_module(f"{code_dir}.cpg_solver")
problem.register_solve(SOLVER, module.cpg_solve)

# test and compile
def solve_cvxpy():
    w.value = W
    problem.solve(solver=SOLVER)
    return x.value

solve_cvxpy().argmax(1)

array([5, 9, 3, 4, 1, 2, 6, 7, 8, 0])

In [18]:
A_ub = np.tile(np.identity(M), (1, N))
b_ub = np.full(M, max_assigned)

A_eq = np.repeat(np.identity(N), M, axis=1)
b_eq = np.full(N, 1.0)

c = W.flatten()

In [19]:
def solve_highs():
    return sp.optimize.linprog(
        c=c,
        A_ub=A_ub,
        b_ub=b_ub,
        A_eq=A_eq,
        b_eq=b_eq,
        method="highs",
    ).x.reshape((N, M))
solve_highs().argmax(1)

array([5, 9, 3, 4, 1, 2, 6, 7, 8, 0])

In [20]:
A_ub_sparse = sp.sparse.csr_matrix(A_ub)
A_eq_sparse = sp.sparse.csr_matrix(A_eq)

In [21]:
%%timeit
solve_cvxpy()

834 μs ± 41.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
%%timeit
solve_highs()

3.79 ms ± 96.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
